In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_file = open("../../data/iyer/train.txt")

In [3]:
train_file_contents = train_file.readlines()

In [4]:
len(train_file_contents)

52997

In [5]:
train = []
for line in train_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        train.append(line.split('\t')[2])

In [6]:
len(train)

52795

In [7]:
len(train[0])

31

In [8]:
len(train[1])

23

In [9]:
import sys
sys.path.append("../../src")
from text_data_utils import *
from os import path
import gensim

In [10]:
train_tok = tokenize_texts(train)

In [11]:
wv = gensim.models.Word2Vec(train_tok).wv

In [12]:
max_len = max(len(text) for text in train_tok)

In [13]:
train_tensor = tokenized_texts_to_tensor(train_tok, wv, max_len)

In [14]:
train_tensor[0, 0]

array([ 1.507817  ,  0.1484023 , -0.20681311,  0.62617654,  0.5449142 ,
       -0.5449318 , -0.21301612,  1.7924953 ,  0.22406252, -0.61348665,
        0.24831563, -1.0640328 ,  0.36666486,  0.04502412,  0.4453507 ,
       -0.5475464 , -0.26597756, -0.48045406,  0.45849183, -0.46563247,
       -0.05305701,  0.5125075 , -0.33755347,  1.2249124 ,  0.28303462,
       -0.70756435, -0.57670695,  0.6581089 ,  0.5804113 , -0.70469946,
       -0.04943128, -1.0484043 ,  0.67638755,  0.88557476,  0.3741503 ,
       -1.1953326 ,  1.3866937 , -0.8176652 ,  0.3666112 ,  0.9233758 ,
        0.21730971,  0.49158496, -0.8107285 ,  0.1139212 ,  0.79562056,
        1.1318586 ,  0.29344016, -0.7276022 ,  0.2629111 ,  0.63989407,
       -0.35141924,  0.14826865, -0.29793808,  0.9161196 ,  0.3829815 ,
        0.66493315,  0.39652708, -0.771028  , -0.43014258, -0.270518  ,
       -0.316743  ,  0.887334  ,  0.34691614, -0.3814799 ,  0.6163675 ,
       -1.131697  , -0.51892763, -0.00366329, -0.26000375,  0.11

In [15]:
train_tensor_fl = np.reshape(train_tensor, (train_tensor.shape[0], train_tensor.shape[1] * train_tensor.shape[2]))

In [16]:
train_tensor_fl.shape

(52795, 3900)

In [17]:
val_file_contents = open('../../data/iyer/valid.txt').readlines()
val = []
for line in val_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        val.append(line.split('\t')[2])
val_tok = tokenize_texts(val)
val_tensor = tokenized_texts_to_tensor(val_tok, wv, max_len)
val_tensor_fl = np.reshape(val_tensor, (val_tensor.shape[0], val_tensor.shape[1] * val_tensor.shape[2]))

In [18]:
from mlp_vae import MLPVariationalAutoEncoder

In [19]:
latent_dim = 256

In [20]:
train_tensor_fl.dtype

dtype('float32')

In [21]:
model = MLPVariationalAutoEncoder(train_tensor_fl.shape[1], latent_dim, final_activation=None)
model.train(train_tensor_fl, val_tensor_fl, 25, 512, tf.keras.optimizers.Adam(learning_rate=0.0001))

Epoch 1 of 25 completed, training loss = 0.14636781811714172, validation loss = 0.0832347571849823
Epoch 2 of 25 completed, training loss = 0.07747786492109299, validation loss = 0.06885802000761032
Epoch 3 of 25 completed, training loss = 0.064786896109581, validation loss = 0.05959059298038483
Epoch 4 of 25 completed, training loss = 0.055100101977586746, validation loss = 0.05074354633688927
Epoch 5 of 25 completed, training loss = 0.04726957157254219, validation loss = 0.04426702857017517
Epoch 6 of 25 completed, training loss = 0.041635360568761826, validation loss = 0.039497680962085724
Epoch 7 of 25 completed, training loss = 0.03742150589823723, validation loss = 0.03579166531562805
Epoch 8 of 25 completed, training loss = 0.03415169194340706, validation loss = 0.03292384371161461
Epoch 9 of 25 completed, training loss = 0.0315316803753376, validation loss = 0.030508067458868027
Epoch 10 of 25 completed, training loss = 0.02935452200472355, validation loss = 0.02858197316527366

In [22]:
import random

In [23]:
random.seed()
random_idx = random.randrange(train_tensor.shape[0])
inp = np.array([train_tensor[random_idx]])

In [24]:
print("(Training Set) Input: ", tensor_to_tokenized_texts(inp, wv)[0])

(Training Set) Input:  ['<s>', 'How', 'can', 'I', 'read', 'the', 'properties', 'of', 'a', 'C', '#', 'class', 'dynamically', '?', '</s>']


In [25]:
rec = np.reshape(model.decode(model.encode(np.array([train_tensor_fl[random_idx]])).sample()), (1, train_tensor.shape[1], train_tensor.shape[2]))

In [26]:
print("(Training Set) Reconstructed: ", tensor_to_tokenized_texts(rec, wv)[0])

(Training Set) Reconstructed:  ['<s>', 'How', 'can', 'I', 'read', 'the', 'properties', 'of', 'a', 'C', '#', 'class', 'dynamically', '?', '</s>']


In [27]:
rec[0, 0]

array([ 1.3962078 ,  0.12766938, -0.2304027 ,  0.6789929 ,  0.55895853,
       -0.57951427, -0.20617224,  1.7013891 ,  0.23728736, -0.61293125,
        0.20292214, -1.0698476 ,  0.3878635 ,  0.03705277,  0.45512006,
       -0.5272546 , -0.31592438, -0.5149551 ,  0.5133568 , -0.461101  ,
       -0.04720134,  0.47412825, -0.33917975,  1.101306  ,  0.26600316,
       -0.6783844 , -0.5514709 ,  0.66507995,  0.58273125, -0.7052709 ,
       -0.00686422, -1.0469769 ,  0.62901074,  0.8767672 ,  0.3506065 ,
       -1.1282785 ,  1.3190396 , -0.7990406 ,  0.2966875 ,  0.90850127,
        0.1168989 ,  0.45771733, -0.809105  ,  0.0999707 ,  0.7554718 ,
        1.0906658 ,  0.3132154 , -0.6970688 ,  0.29668817,  0.5814272 ,
       -0.35530728,  0.14546826, -0.2444585 ,  0.93830204,  0.42566013,
        0.68332213,  0.40024167, -0.79780513, -0.47994983, -0.28567576,
       -0.27960265,  0.89332455,  0.34961796, -0.33336848,  0.6311755 ,
       -0.98190343, -0.509345  , -0.0158149 , -0.2633227 ,  0.12

In [28]:
inp[0, 0]

array([ 1.507817  ,  0.1484023 , -0.20681311,  0.62617654,  0.5449142 ,
       -0.5449318 , -0.21301612,  1.7924953 ,  0.22406252, -0.61348665,
        0.24831563, -1.0640328 ,  0.36666486,  0.04502412,  0.4453507 ,
       -0.5475464 , -0.26597756, -0.48045406,  0.45849183, -0.46563247,
       -0.05305701,  0.5125075 , -0.33755347,  1.2249124 ,  0.28303462,
       -0.70756435, -0.57670695,  0.6581089 ,  0.5804113 , -0.70469946,
       -0.04943128, -1.0484043 ,  0.67638755,  0.88557476,  0.3741503 ,
       -1.1953326 ,  1.3866937 , -0.8176652 ,  0.3666112 ,  0.9233758 ,
        0.21730971,  0.49158496, -0.8107285 ,  0.1139212 ,  0.79562056,
        1.1318586 ,  0.29344016, -0.7276022 ,  0.2629111 ,  0.63989407,
       -0.35141924,  0.14826865, -0.29793808,  0.9161196 ,  0.3829815 ,
        0.66493315,  0.39652708, -0.771028  , -0.43014258, -0.270518  ,
       -0.316743  ,  0.887334  ,  0.34691614, -0.3814799 ,  0.6163675 ,
       -1.131697  , -0.51892763, -0.00366329, -0.26000375,  0.11

In [29]:
test_file_contents = open('../../data/iyer/test.txt').readlines()
test = []
for line in test_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        test.append(line.split('\t')[2])
test_tok = tokenize_texts(test)
test_tensor = tokenized_texts_to_tensor(test_tok, wv, max_len)
test_tensor_fl = np.reshape(test_tensor, (test_tensor.shape[0], test_tensor.shape[1] * test_tensor.shape[2]))

In [30]:
random.seed()
random_idx = random.randrange(test_tensor.shape[0])

In [31]:
print("(Test Set) Input: ", tensor_to_tokenized_texts(np.array([test_tensor[random_idx]]), wv)[0])

(Test Set) Input:  ['<s>', 'help', 'with', 'a', 'two', 'table', 'linq', 'query', '</s>']


In [32]:
print("(Test Set) Reconstructed: ", tensor_to_tokenized_texts(np.reshape(model.decode(model.encode(np.array([test_tensor_fl[random_idx]])).sample()), (1, test_tensor.shape[1], test_tensor.shape[2])), wv)[0])

(Test Set) Reconstructed:  ['<s>', 'help', 'with', 'a', 'two', 'table', 'linq', 'query', '</s>']
